# 协程

- 适合处理需要等待的任务
- asyncio创建event_loop包含多个task决定执行那个任务
    - loop = asyncio.get_event_loop()
    - 同时执行的只能有一个，没有`竞争冒险`的问题
- coroutine 
    - coroutine function: async def
    - coroutine object: 调用coroutine function返回的对象，并不会运行coroutine的函数
- 如何运行coroutine？
    - 进入async模式:
        -  入口：asyncio.run(main()) 
            1. 创建event_loop
            2. 把main()注册到event_loop中
    - 在async模式下增加task
        1. `await func()`  func 由async def定义，返回一个coroutine对象
            1. 将coroutine包装成task并注册到event_loop中
            2. task的依赖关系，task依赖于func完成才能继续
            3. yield直到event_loop再次运行到task，把func的返回值返回
            - 相当于一个async def定义的函数生成了一个func和一个task，task依赖于func
        2. `create_task函数`，先注册多个task，再决定运行那些task
            - task = asyncio.create_task(func()) func也是一个coroutine函数由async def定义
            - `await task`当await时，不需要注册task2，告诉event_loop需要task1完成，控制权交还给event_loop，当task1完成后获取返回值
        3. 获取所有task的返回值future
            - `await asyncio.gather(ls)`， ls可以是若干个coroutine，也可以是若干个task，或者future(多层gather)
            - gather对task和corotine都会一次性注册所有task到event_loop中，此时定义corotine更方便


In [1]:
import asyncio
import time

async def main():
    print('start', time.strftime('%X'))
    await asyncio.sleep(2)
    await asyncio.sleep(4) # 需要等到上面的完成后才能变成task
    print('end', time.strftime('%X'))

await main() # 6s

start 23:05:06
end 23:05:12


In [2]:
import asyncio
import time

async def main():
    task1 = asyncio.create_task(asyncio.sleep(2))
    task2 = asyncio.create_task(asyncio.sleep(4))

    print('start', time.strftime('%X'))
    await task1
    await task2  # 先注册task1，task2，当await task1时，此时task2也会被执行
    print('end', time.strftime('%X'))

await main() # 4s

start 23:05:12
end 23:05:16


In [3]:
import asyncio
import time


async def main():

    # multi coroutine
    start = time.strftime('%X')
    ret = await asyncio.gather(asyncio.sleep(2), asyncio.sleep(4)) # 此时会一次性注册所有task，然后一起执行
    end = time.strftime('%X')
    print(start, end, ret) # 4s

    # multi task
    start = time.strftime('%X')
    task1 = asyncio.create_task(asyncio.sleep(2))
    task2 = asyncio.create_task(asyncio.sleep(4))

    ret = await asyncio.gather(task1,task2)
    end = time.strftime('%X')
    print(start, end, ret) # 4s 

await main() # 

23:05:17 23:05:21 [None, None]
23:05:21 23:05:25 [None, None]


In [4]:
async def sleep():
    await asyncio.sleep(2)
    return 'sleep'

async def main():
    ret = await asyncio.gather(*[sleep() for _ in range(5)])
    print(ret)

await main() # 2s 

['sleep', 'sleep', 'sleep', 'sleep', 'sleep']


In [5]:
async def main():
    await sleep()
    await sleep()
    await sleep()
    await sleep()
    await sleep()

await main() # 10s 